In [ ]:
import random
import numpy as np
import pandas as pd

In [ ]:
from Farm_Evaluator import getTurbLoc, loadPowerCurve, binWindResourceData, preProcessing, getAEP, checkConstraints

In [ ]:
power_curve   =  loadPowerCurve('power_curve.csv')
turb_rad = 50

In [ ]:
n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t = preProcessing(power_curve)

In [ ]:
# Years on which the data is available
year_list = ['2007','2008','2009','2013','2014','2015','2017']
wind_data_list = []
for year in year_list:
    wind_data_list.append(pd.read_csv("wind_data_{}.csv".format(year)))

In [ ]:
# Problem parameter
min_lim = 50
max_lim = 3950
turb_count = 50
penalty = 2

In [ ]:
def gen_coord():
    x = round(random.uniform(min_lim, max_lim),2)
    y = round(random.uniform(min_lim, max_lim),2)
    return x,y

In [ ]:
def check_violation(turb1, turb2):
    return np.linalg.norm(turb1 - turb2) > 8*turb_rad

In [ ]:
def generate_turb_locations():
    arr = [gen_coord()]
    for i in range(turb_count-1):
        while True:
            new_cord = gen_coord()

            if all([check_violation(np.array(cord), np.array(new_cord)) for cord in arr]):
                arr.append(new_cord)
                break
    return arr

In [ ]:
def calculate_AEP(turb_coords):
    aep = []
    for wind_data in wind_data_list:
        wind_inst_freq =  binWindResourceData(wind_data)
        aep.append(getAEP(turb_rad, turb_coords, power_curve, wind_inst_freq, 
                  n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t))
    return sum(aep)/len(aep)

In [ ]:
class TurbGroup:
    def __init__(self):
        self.turb_coords = np.array(generate_turb_locations())
        self.fitness = -1
        self.violation_count = 0
    
    def count_violation(self):
        for i,turb1 in enumerate(self.turb_coords):
            for turb2 in np.delete(self.turb_coords, i, axis=0):
                if  np.linalg.norm(turb1 - turb2) < 8*turb_rad:
                    self.violation_count += 1
                    
        
    def calculate_fitness(self):
        self.count_violation()
        fitness = calculate_AEP(self.turb_coords)
        fitness -= fitness*penalty*self.violation_count
        self.fitness = fitness


In [ ]:
# GA parameters
POP_SIZE = 500
GEN_COUNT = 100
TOURNAMENT_SIZE = 3
MUT_PROB = 0.9 #0.15
XO_PROB = 0.75

In [ ]:
def ga():

    pop = init_pop(POP_SIZE)
    pop = fitness(pop)
      
    for generation in range(GEN_COUNT):    
        nextgen_pop = []
        for i in range(int(POP_SIZE/2)):
            parent1 = selection(pop)
            parent2 = selection(pop)
            offspring = crossover(parent1, parent2)
            nextgen_pop.append(mutation(offspring[0]))
            nextgen_pop.append(mutation(offspring[1]))
        pop = fitness(nextgen_pop)
        best = sorted(pop, key=lambda x: x.fitness, reverse=True)
        print("Best fit in Generation no {}: {}".format(generation, best[0].fitness))
    return best[0]


In [ ]:
def init_pop(population_count):
    return [TurbGroup() for _ in range(population_count)]

In [ ]:
def fitness(population):
    for agent in population:
        agent.calculate_fitness()
    return population

In [ ]:
def selection(population):
    players = random.sample(population, TOURNAMENT_SIZE)
    sorted_players = sorted(players, key=lambda x: x.fitness, reverse=True)

    return sorted_players[0]

In [ ]:
def crossover(p1, p2):
    if random.random() < XO_PROB:
        xo_pt = random.randrange(turb_count)
        p1.turb_coords[:xo_pt],p2.turb_coords[:xo_pt] = p1.turb_coords[:xo_pt],p2.turb_coords[:xo_pt]

    return p1, p2

In [ ]:
def mutation(agent):
    if random.random() < MUT_PROB:
        return TurbGroup()
    else:
        return agent

In [ ]:
# best_fit is an instance of the class TurbGroup
# you can get the location using the attribute turb_coords
# eg: best_fit.turb_coords
best_fit = ga()

Best fit in Generation no 0: 508.8590207781111
Best fit in Generation no 1: 509.35512581961495
Best fit in Generation no 2: 508.82677361624576
Best fit in Generation no 3: 509.1528208269392
Best fit in Generation no 4: 509.1528208269392
Best fit in Generation no 5: 508.9628991263253
Best fit in Generation no 6: 508.9940530831474
Best fit in Generation no 7: 508.67769672938755
Best fit in Generation no 8: 508.70463540213444
Best fit in Generation no 9: 508.85152110508506
Best fit in Generation no 10: 508.821417388916
Best fit in Generation no 11: 509.9419621058873
Best fit in Generation no 12: 509.4889646693638
Best fit in Generation no 13: 510.0104807608468
Best fit in Generation no 14: 509.5754949515207
Best fit in Generation no 15: 509.4889646693638
Best fit in Generation no 16: 508.89020815168107
Best fit in Generation no 17: 509.4831310599191
Best fit in Generation no 18: 509.44104505266466
Best fit in Generation no 19: 509.12712812151227
Best fit in Generation no 20: 509.209271262

In [ ]:
best_fit.turb_coords

array([[2811.74,  465.4 ],
       [2527.14, 2873.87],
       [1745.46, 1517.64],
       [3412.27, 1853.8 ],
       [3152.48, 2446.51],
       [2308.61, 1792.03],
       [3365.2 , 3511.55],
       [ 528.25, 1053.7 ],
       [1038.75, 2650.12],
       [3740.04, 3240.75],
       [1074.61, 1616.64],
       [ 162.82, 3437.37],
       [3937.64,   52.34],
       [2514.68,   80.49],
       [1712.37, 2085.23],
       [3358.66,   56.73],
       [2238.06, 3873.95],
       [2942.21, 3751.52],
       [1814.44,  674.22],
       [ 983.64, 3420.39],
       [3785.91, 1295.37],
       [2779.58, 1715.29],
       [1519.87, 2876.45],
       [3244.71,  994.24],
       [ 393.57, 1711.2 ],
       [ 746.28, 3844.16],
       [3835.23,  455.06],
       [3874.31,  858.41],
       [2477.27, 3468.6 ],
       [3731.04, 3686.96],
       [ 588.4 , 2258.91],
       [2001.85,   53.08],
       [2771.04, 1192.55],
       [3690.24, 2194.75],
       [ 249.86,  729.83],
       [1153.94,  566.76],
       [1567.53, 3664.33],
 

In [ ]:
checkConstraints(best_fit.turb_coords, turb_rad*2)

Both perimeter and proximity constraints are satisfied !!



()

In [ ]:
count = 50
for _ in range(count):
    locs = np.array(generate_turb_locations())
    plt.scatter(locs[:,0],locs[:,1])
plt.scatter(best_fit.turb_coords[:,0],best_fit.turb_coords[:,1], c='black')

NameError: ignored

In [ ]:
import pandas as pd
loc=pd.DataFrame(best_fit.turb_coords)
loc.to_csv('MAIN.csv')